# 2023 CITS4012 Assignment
*Make sure you change the file name with your student id.*

# Readme

Cetain libraries needs to be installed for the whole code to work. Please make sure to run the below cmd functions. before running the full code.

In [91]:
# Installing spacy for Named Entity Tagging
!pip install spacy

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
# Authenticate and create the PyDrive client.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [114]:
# To Tabulate the values
!pip install tabulate

In [92]:
# Downloading the pre-trained NLP Model for Named Entity Tagging
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


#### Importing Libararies

In [130]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/naufaln/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/naufaln/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/naufaln/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/naufaln/nltk_data...


True

In [297]:
import pprint
import re
import math

# For parsing our XML data
from lxml import etree 
import numpy as np
import pandas as pd
from tabulate import tabulate
from statistics import median

# For data processing
import nltk
import gensim.downloader as api
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.preprocessing.text import Tokenizer

# For Named Enity Tagging
import spacy
import en_core_web_sm
from spacy import displacy
from collections import Counter

# importing necessary libraries for TF-IDF
from nltk.tokenize import TreebankWordTokenizer

# For Modelling
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#### Importing Training and Testing Data

In [2]:
# importing Training and Testing Data
training_data = pd.read_csv('./Data/WikiQA-train.tsv', sep='\t')
test_data = pd.read_csv('./Data/WikiQA-test.tsv', sep='\t')

In [120]:
print(training_data.columns)

Index(['QuestionID', 'Question', 'DocumentID', 'DocumentTitle', 'SentenceID',
       'Sentence', 'Label'],
      dtype='object')


# 1.DataSet Processing

##### Function to convert the given data to required format

In [7]:
def shrink_columns(df):
    # Create a new dataframe with four columns
    new_df = pd.DataFrame(columns=['QuestionID', 'Question', 'Document', 'Answer'])

    # Loop through the unique QuestionIDs in the original dataframe
    for qid in df['QuestionID'].unique():
        # Get the first question associated with this QuestionID
        first_question = df.loc[df['QuestionID'] == qid, 'Question'].iloc[0]
        
        # Get all sentences associated with this QuestionID
        sentences = df.loc[df['QuestionID'] == qid, 'Sentence']
        
        # Concatenate all sentences into a single string
        concatenated_sentence = ' '.join(sentences)
        
        # Get the sentence associated with this QuestionID where the Label is 1
        answer = df.loc[(df['QuestionID'] == qid) & (df['Label'] == 1), 'Sentence']
        
        # If there is at least one such row, get the first sentence
        if not answer.empty:
            answer = answer.iloc[0]
        else:
            answer = ''
        
        # Add the QuestionID, first_question, concatenated_sentence, and answer to the new dataframe
        new_row = {'QuestionID': qid, 'Question': first_question, 'Document': concatenated_sentence, 'Answer': answer}
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

    return new_df

In [8]:
shrinked_training_data = shrink_columns(training_data)
shrinked_test_data = shrink_columns(test_data)

# Print the new dataframe
shrinked_test_data.head(16)


,QuestionID,Question,Document,Answer
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African immigration to the United States refer...,"As such, African immigrants are to be distingu..."
1,Q3,how large were early jails,"A prison (from Old French prisoun), also known...",
2,Q4,how a water pump works,"A small, electrically powered pump A large, el...",Pumps operate by some mechanism (typically rec...
3,Q20,how old was sue lyon when she made lolita,Lolita is a 1962 comedy-drama film by Stanley ...,"The actress who played Lolita, Sue Lyon , was ..."
4,Q33,how are antibodies used in,Each antibody binds to a specific antigen ; an...,"An antibody (Ab), also known as an immunoglobu..."
5,Q47,how old is alicia in 2009,"Alicia Augello Cook (born January 25, 1981), k...",
6,Q49,how big is a land section generally,Sectioning a township . Perfectly square full ...,
7,Q54,how old was shakespeare's juliet,Juliet is one of the title characters in Willi...,
8,Q57,how are fire bricks made,Refractory bricks in a torpedo car used for ha...,
9,Q59,HOW MUCH IS CENTAVOS IN MEXICO,The peso ( sign : $; code : MXN) is the curren...,"The peso is subdivided into 100 centavos, repr..."


In [9]:
shrinked_training_data['labels'] = ''

for i in range(len(shrinked_training_data)):
    paragraph = shrinked_training_data.loc[i, 'Document']
    token = [None for i in range(len(paragraph))]
    if shrinked_training_data.loc[i, 'Answer'] != '':
        start_index = paragraph.find(shrinked_training_data.loc[i, 'Answer'])
        end_index = start_index + len(shrinked_training_data.loc[i, 'Answer'])
        token[start_index] = 'S'
        for j in range(start_index+1, end_index):
            token[j] = 'I'
        token[end_index-1] = 'E'
    shrinked_training_data.at[i, 'labels'] = token

# 2.QA Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

#### Tokenization

In [268]:
def tokenize(sentance):
    sent_text=[]
    content_text = re.sub(r'\([^)]*\)', '', sentance.lower())
    sent_text.extend(word_tokenize(content_text))


    #commenting it in doubt whether to remove punctuations or not
    '''
    # Removing punctuation and changing all characters to lower case
    normalized_text = []
    for string in sent_text:
        tokens = re.sub(r"[^a-z0-9.]+", " ", string.lower())
        normalized_text.append(tokens)
    
    print(sent_text)
    '''
    return sent_text


#### Tokenising Sequences

In [ ]:
def tokenize_list(sequences):
    tokenized_list = []
    for seq in sequences:
        tokenized_list.append(tokenize(seq))

    return tokenized_list

#### Word Embedding

In [153]:
def word_embedding(tokens):
    word_embed_model = api.load("glove-twitter-25")
    # Create the Embedding lookup table for the first pre-trained embedding
    emb_dim = word_embed_model.vector_size

    emb_table = []
    for i, word in enumerate(tokens):
        if str(word) in word_embed_model:
            emb_table.append(word_embed_model[str(word)])
        else:
            emb_table.append([0]*emb_dim)
    emb_table = np.array(emb_table)
        
    return emb_table

#### Average Length of Sequences

In [289]:
def get_average_length(sequences):
    list_of_lengths = []
    avg_length = 0
    for seq in sequences:
        list_of_lengths.append(len(seq))
    
    avg_length = round(sum(list_of_lengths)/len(list_of_lengths))
    return avg_length

#### Padding Sequences

In [301]:
def pad_sequences(sequences):
    # Find the max length of the sequences
    max_length = max(len(seq) for seq in sequences)
    
    # Pad the sequences based on the max length
    padded_sequences = []

    for seq in sequences:
        num_padding = max_length - len(seq)
        padded_seq = seq + ['[PAD]'] * num_padding
        padded_sequences.append(padded_seq)
    
    return padded_sequences

### Questions Pre-Processing

In [ ]:
# Getting the list of questions
qn_list = shrinked_training_data['Question']

##### Getting full list of words in Questions

In [167]:
def get_Q_full_word_list(sentance_list):
    Q_full_word_list = list()
    for Q in sentance_list:
        Q = Q.lower()
        Q_word_list = tokenize(Q)
        Q_full_word_list.append(Q_word_list)
    return(Q_full_word_list)

#### Word to Index

In [ ]:
Q_full_word_list = get_Q_full_word_list(qn_list)

# set up a vocab to index dictionary
Q_word_to_ix = {"<BOS>": 0, "<EOS>":1} 
for sentence in Q_full_word_list:
    for word in sentence:
        if word not in Q_word_to_ix:
            Q_word_to_ix[word] = len(Q_word_to_ix)
word_list = list(Q_word_to_ix.keys())

#### Padding the Questions with Max length

In [304]:
tokenised_qn_list = tokenize_list(qn_list)
padded_qn_list = pad_sequences(tokenised_qn_list)

In [307]:
input_questions = pd.DataFrame()
for i in range(2):
    input_questions["Words"] = padded_qn_list[i]
    
    # Word Embedding
    # generating word embedding values 
    qn_emb_table = word_embedding(padded_qn_list[i])
    qn_emb_table = pd.DataFrame(qn_emb_table)
    input_questions = pd.concat([input_questions, qn_emb_table], axis=1)

print(input_questions)
    

         Words        0         1         2         3         4        5  \
0          how  0.50363  0.257190 -0.160620 -0.065956 -0.300250 -0.04945   
1          are  0.18660 -0.098326 -0.122680 -0.938220 -0.401610  0.63830   
2          the -1.22180 -1.043200 -0.096553 -0.041382 -1.586400 -0.16366   
3   directions -0.70318 -1.149900  0.181350 -0.075280 -1.242100  0.15105   
4           of -0.57660 -0.007180 -0.543770 -0.792860 -0.259320 -0.82164   
5          the  1.10400 -0.346290  0.088792 -0.255400 -0.023462  0.51487   
6     velocity  0.00000  0.000000  0.000000  0.000000  0.000000  0.00000   
7          and  0.00000  0.000000  0.000000  0.000000  0.000000  0.00000   
8        force  0.00000  0.000000  0.000000  0.000000  0.000000  0.00000   
9      vectors  0.00000  0.000000  0.000000  0.000000  0.000000  0.00000   
10     related  0.00000  0.000000  0.000000  0.000000  0.000000  0.00000   
11          in  0.00000  0.000000  0.000000  0.000000  0.000000  0.00000   
12          

### Document Pre-processing

#### Labelling

In [195]:
def generate_labels(document, answer):
    labels = ["Not Answer" for i in range(len(document))]
    if answer != '':
        #start_index = document.index(answer)
        start_index = [i for i in range(len(document)-len(answer)+1) if document[i:i+len(answer)] == answer]
        print(start_index)
        start_index = start_index[0]
        end_index = start_index + len(answer)
        labels[start_index] = 'Start'
        for j in range(start_index+1, end_index):
            labels[j] = 'Middle'
        labels[end_index-1] = 'End'
    return labels

#### TF_IDF

In [99]:
def tf_idf(tokens):
    tf_idf_list = list()
    tknzr = TreebankWordTokenizer()
    DF = {}

    # get each unique word in the doc - and count the number of occurrences in the document
    for term in np.unique(tokens):
        try:
            DF[term] +=1
        except:
            DF[term] =1

    tf_idf = []
    N = len(tokens) 
    doc_id = 0
    counter = Counter(tokens)
    total_num_words = len(tokens) 
    for term in tokens:
        tf = counter[term]/total_num_words
        df = DF[term]
        idf = math.log(N/(df+1))+1
        tf_idf.append(tf*idf)

    doc_id += 1
    tf_idf_list.append(tf_idf)

    return tf_idf_list[0]

#### POS tagging

In [110]:
def pos_tagging(tokens):
    tagged_words = pos_tag(tokens)
    tagged_words_list, tags_list = zip(*tagged_words)
    return tags_list

#### Named Entity Tagging

In [216]:
def NE_Tagging(document):
    NE_Tag_table = []
    tokens = []
    # loading pre-trained model of NER
    entity_tagging_model = en_core_web_sm.load()
    article = entity_tagging_model(document)
    sentences = [x for x in article.sents]
    for sentence in sentences:
        for word in sentence:
            NE_Tag_table.append(str(word.ent_type_))
            tokens.append(str(word).lower())
    for i in range(len(NE_Tag_table)):
        if(NE_Tag_table[i] == ''):
            NE_Tag_table[i] = "O"

    return tokens, NE_Tag_table

#### Lemmatization

In [182]:
# function to get the wordnet POS tag and convert to use with lemmatizer
def get_wordnet_pos(tags):
    if tags.startswith('J'):
        return 'a'  # Adjective
    elif tags.startswith('V'):
        return 'v'  # Verb
    elif tags.startswith('N'):
        return 'n'  # Noun
    elif tags.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'

In [183]:
def lemmatization(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmitized = [lemmatizer.lemmatize(token, pos=get_wordnet_pos(tag)) for token,tag in zip(tokens['Words'], tokens['POS Tags']) ]  
    return lemmitized

In [247]:
doc_list = shrinked_training_data['Document']


# Delete later if not needed

In [222]:
def question_padding(questions):
    # Tokenize the questions
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(questions)
    question_tokenised = tokenizer.texts_to_sequences(questions)

    # Get the maximum length of all the questions in the 'Question' column of the dataframe
    max_length = max(len(question) for question in question_tokenised)

    # Pad the sequence based on the max length defined, to make sure the list of sequences are of same length
    def pad_sequence(seq_list, max_length, index_dict):
        res = []
        for seq in seq_list:
            temp = seq[:]
            # Pad the sequence with zeros if its length is less than max_length
            temp += [index_dict['[PAD]']] * (max_length - len(seq))
            res.append(temp)
        return np.array(res)

    # Add the Question_padded column with the padded sequences
    word2index = tokenizer.word_index
    word2index['[PAD]'] = 0 # Add the '[PAD]' token to the word2index dictionary
    questions_padded = pad_sequence(question_tokenised, max_length, word2index)
    print(tabulate(questions_padded, headers='firstrow', tablefmt='fancy_grid'))


#### Training

In [308]:
D_full_word_list = list()
for i in range(1):
    doc = shrinked_training_data.loc[i, "Document"]

    answer = tokenize(shrinked_training_data.loc[i, 'Answer'])

    doc_tokens = tokenize(doc)

    word_features = pd.DataFrame()

    # 1. Words
    word_list, NE_Tag_list = NE_Tagging(doc)
    word_features["Words"] = word_list

    # 2. Labels
    # generating Labels to identify the position of the Answer in the document
    labels = generate_labels(word_list, answer)
    word_features["Labels"] = labels

    # Creating a dictionary of Words in Documents
    D_full_word_list.append(word_list)

    # 3. POS tagging
    pos_tags = pos_tagging(word_list)
    word_features["POS Tags"] = pos_tags

    # 4. Lemmetization
    word_list = lemmatization(word_features)
    word_features["Words"] = word_list
    
    # 5. Named Entity Tags
    word_features["Named Entity Tags"] = NE_Tag_list
    
    # 7. TF_IDF
    tf_idf_list = tf_idf(word_list)
    word_features["TF-IDF"] = tf_idf_list

    # 8. Word Embedding
    # generating word embedding values 
    emb_table = word_embedding(word_list)
    
    emb_table = pd.DataFrame(emb_table)

    word_features = pd.concat([word_features, emb_table], axis=1)

    print(word_features)
    

[25]
       Words      Labels POS Tags Named Entity Tags    TF-IDF         0  \
0          a  Not Answer       DT                 O  0.279108  0.212940   
1     partly  Not Answer       RB                 O  0.069777 -0.980010   
2   submerge  Not Answer      VBN                 O  0.069777 -1.125800   
3    glacier  Not Answer       NN                 O  0.418663 -1.221800   
4       cave  Not Answer       NN                 O  0.488440 -1.287700   
..       ...         ...      ...               ...       ...       ...   
59      year  Not Answer       NN              DATE  0.069777 -0.049064   
60         -  Not Answer        :              DATE  0.069777  0.771700   
61     round  Not Answer       NN              DATE  0.069777 -0.746810   
62       ice  Not Answer       NN                 O  0.348886 -0.768120   
63         .  Not Answer        .                 O  0.209331  0.695860   

          1         2         3         4  ...        15       16       17  \
0   0.31035  0.1

#### Question Model

#### Training

##### Question Summary

# 3.Model Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.1. Input Embedding Ablation Study

(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 



###3.2. Attention Ablation Study
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.3. Hyper Parameter Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 